<p style="font-size: 2rem; margin-bottom: 0">Design Patterns 1.0.1</p>
(**Caution:** May contain Java)

# First an example
*(Observer Pattern a.k.a publisher subscriber)*

**Problem:** We have a `Gps` class that periodically reads position from a sensor.  A map and distance tracker need to be updated with position

In [51]:
class Position:
    def __init__(self, lat, long):
        self.lat = lat
        self.long = long

class Sensor:
    def read():
        # Or whatever the current coordinates are
        return 39.97, -83.059

class Gps:
    def __init__(self):
        self.sensor = Sensor()
        self.position = Position(0, 0, 0)
    def main_loop(self):
        while True:
            self.position = self.sensor.read()
    def get_position(self):
        return self.position

In [52]:
# 1st Attempt: have Map and Distance periodically check for updates
from time import sleep
class Map:
    """A class which displays the map"""
    def __init__(self, gps):
        self.gps = gps
    def main_loop(self):
        while True:
            position = Gps.get_position()
            # update map
            sleep(0.1)  # wait a 1/10th of a second
            
class Distance:
    pass  # would be set up the same way

![Polling](polling.svg)

![Push](push.svg)

In [53]:
class Gps:
    def __init__(self):
        self.sensor = Sensor()
        self.map = Map()
        self.distance = Map()
    def main_loop(self):
        while True:
            position = self.sensor.read()
            self.map.update(position)
            self.distance.update(position)
    
class Map:
    """A class which displays the map"""
    def update(self, position):
        pass # update map
            
class Distance:
    """A class to track total distance"""
    def update(self, position):
        pass # add to total distance

In [54]:
# Now the client wants more features
class Waypoints:
    def update(self, position):
        pass  # update waypoints
    
class Speed:
    def update(self, position):
        pass  # update current and average speed

In [55]:
class Gps:
    """In the parlance of the observer pattern Gps is 
    the observable and Map, Distance, Waypoints and 
    Speed are observers """
    def __init__(self):
        self.sensor = Sensor()
        self.observers = []
    def main_loop():
        while true:
            position = self.sensor.read()
            for observer in self.observers:
                observer.update(position)
                
def start_gps(waypoints_enabled=False):
    gps = Gps()
    for observer in [Waypoints(), Distance(), Speed()]:
        gps.observers.append(observer)
    if waypoints_enabled:
        gps.observers.append(Waypoints())

# Loose Coupling

The principal that objects should be able to interact with minimal knowledge of each other

# What are patterns anyway?

1. Provide receipes for solving common problems
1. Define a vocabulary for discussing software components

# Creational Patterns

For creating instances of classes

## Singleton Pattern

Goal is to have 1 instance of a particular object in your application.  Common use case is to store the settings of your application.  Django's `settings` object is a good example:

```python
from django.conf import settings
from_email = settings.DEFAULT_FROM_EMAIL
```

In [56]:
# in config.py module of some app

class Settings:
    def __init__(self):
        pass  # load settings from ~/.myapprc
    

_settings = Settings()
def get_settings():
    return _settings

## Builder Pattern

Goal is to make it easer to set up complex object composed of other objects

In this example we are building houses with a variable number of windows, doors, and rooms.  Additionaly houses can have garages, swimming pools, and gardens [2]

In [57]:
# First some classes for the optional features
class Garage:
    pass

class SwimmingPool:
    pass

class Garden:
    pass

In [58]:
class House1:
    def __init__(
        self, windows: int, doors: int, rooms: int,
        has_garage: bool = False,
        has_swimming_pool: bool = False,
        has_garden: bool = False,
    ):
        self.windows = windows
        self.door = doors
        self.rooms = rooms
        self.garage = Garage() if has_garage else None
        self.swimming_pool = SwimmingPool() if has_swimming_pool else None
        self.garden = Garden if has_garden else None

Java Constructor (`__init__`) calls

```java
House1 house1= new House1(4, 4, 4, false, false, true)
House1 hosue2 = new House1(4, 2, 4, true, true, true)
```

In [59]:
house1 = House1(4, 4, 4, has_garden=True)
house2 = House1(4, 2, 4, has_swimming_pool=True)

In [60]:
class House2:
    def __init__(self):
        self.windows = None
        self.door = None
        self.rooms = None
        self.garage = None
        self.swimming_pool = None
        self.garden = None

In [61]:
class HouseBuilder:
    # Defined HOUSE_CLASS so I don't need to re-define __init__ and reset in sub-classes
    HOUSE_CLASS = House2
    def __init__(self):
        self.house = self.HOUSE_CLASS()
    def reset(self):
        self.house = self.HOUSE_CLASS()
    def build_walls(self):
        pass
    def build_doors(self, doors):
        pass
    def build_windows(self, windows):
        pass
    def build_roof(self):
        pass
    def build_garage(self):
        pass

In [62]:
class FancyHouse(House2):
    pass

class FancyHouseBuilder(HouseBuilder):
    HOUSE_CLASS = FancyHouse
    def build_walls(self):
        pass  # build fancy walls
    def build_roof(self):
        pass  # build fancy roof

In [63]:
class SimpleHouse(House2):
    pass

class SimpleHouseBuilder(HouseBuilder):
    HOUSE_CLASS = SimpleHouse
    def build_garage(self):
        pass  # builds a carport instead

In [64]:
class Director:
    BUILDERS = {
        House2: HouseBuilder(),
        FancyHouse: FancyHouseBuilder(),
        SimpleHouse: SimpleHouseBuilder(),
    }
    def make(self, house_class, doors, windows):
        builder = self.BUILDERS[house_class]
        builder.reset()
        builder.build_walls()
        builder.build_doors(doors)
        builder.build_windows(windows)
        builder.build_roof()
        builder.build_garage()    

In [65]:
# Example of a builder in the lxml library [3]
from lxml.builder import E

doc = E('body', 
        E('h1', 'The Title'), 
        E('p', 'Always write Python'))

## Prototype Pattern

Goal create new objects from existing objects.  In Java/C# you might create a Prototype `interface` stipulating that implementing classes should have a `clone()` method.  In Python we use `copy` and `deepcopy` from Pythons `copy` standard library module.

In [66]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
class Triangle:
    def __init__(self, p1, p2, p3):
        self.p1 = p1
        self.p2 = p2
        self.p3 = p3

t1 = Triangle(Point(0,0), Point(3,0), Point(3,4))

In [67]:
# This shows the memory address of the objects
def show_ids(triangle):
    print(f"t  {id(triangle):,}")
    print(f"p1 {id(triangle.p1):,}")
    print(f"p2 {id(triangle.p2):,}")
    print(f"p3 {id(triangle.p3):,}")

In [68]:
from copy import copy
show_ids(t1)
t2 = copy(t1)
print()
show_ids(t2)

t  139,999,159,879,232
p1 139,999,156,720,064
p2 139,999,156,722,224
p3 139,999,159,880,672

t  139,998,816,296,000
p1 139,999,156,720,064
p2 139,999,156,722,224
p3 139,999,159,880,672


In [69]:
from copy import deepcopy
show_ids(t1)
t3 = deepcopy(t1)
print()
show_ids(t3)

t  139,999,159,879,232
p1 139,999,156,720,064
p2 139,999,156,722,224
p3 139,999,159,880,672

t  139,999,159,938,640
p1 139,999,159,936,048
p2 139,999,159,939,024
p3 139,999,158,310,896


# Structural Patterns

# Behavioral Patterns

Define how classes behave and interact

## Iterator Pattern

Goal: To access all the items in a data structure (`list`, `set`, `dict`, tree, queue, graph, etc)

This is baked into Python and the standard library

In Python:

<table>
  <thead>
    <tr>
      <td>Object Type</td>
      <td>Has Special Method</td>
      <td>Corresponding built-in function</td>  
    </tr>    
  </thead>
  <tbody>
    <tr>
      <td>iterable</td>  
      <td><code>__iter__()</code></td>
      <td><code>iter()</code></td>
    </tr>
    <tr>
      <td>iterator</td>  
      <td><code>__next__()</code></td>
      <td><code>next()</code></td>
    </tr>
  </tbody>      
</table>

In [74]:
# Loop through a list (the hard way)
a = [0, 1, 2]
itr = iter(a)
while True:
    try:
        value = next(itr)
    except StopIteration:
        break
    else:
        print(value)
        
# Python does this automagically when you do this:
# for value in a:
#     print(value)

0
1
2


# Resources

* Brandon Rhodes Design Patterns 1 (start of video is repeated, start at 17 min)
  * https://pyvideo.org/pyohio-2012/python-design-patterns-1.html
  * https://www.youtube.com/watch?v=Er5K_nR5lDQ
* https://python-patterns.guide/
* https://refactoring.guru
* https://diagrams.mingrammer.com/
* https://docs.oracle.com/javase/tutorial/java/javaOO/classes.html


# Footnotes

1. https://refactoring.guru/design-patterns/structural-patterns
1. https://refactoring.guru/design-patterns/builder
1. https://www.youtube.com/watch?v=Er5K_nR5lDQ [40 minutes]